In [3]:
# Code Block 1

# Importing Libraries required for NLP
import nltk
nltk.download("punkt")
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
# Importing Libraries needed for Tensorflow processing
import tensorflow as tf   # version 1.13.2
import numpy as np
import tflearn            # version 0.3.2
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/l

### Block 1 code description
In the above code block we are importing the tools that are necessary to process text data and make a model to learn from that
text data in order develop contextual learning. We will be needing the tools/packages namely NLTK, Tensorflow, Numpy, Random, TFlearn, json to achieve our objective.

In [2]:
#!pip install tflearn==0.3.2
#!pip install tensorflow==1.13.2

In [4]:
# Code block 2 
# importing our intent file, which will be used for training the model.
with open("intents.json") as json_data: 
    intents = json.load(json_data)      # Loading data from intents.json file to variable intents

In [8]:
intents['intents'][0:3]

[{'tag': 'greeting',
  'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
  'responses': ['Hello, thanks for visiting',
   'Good to see you again',
   'Hi there, how can I help?'],
  'context_set': ''},
 {'tag': 'goodbye',
  'patterns': ['Bye', 'See you later', 'Goodbye'],
  'responses': ['See you later, thanks for visiting',
   'Have a nice day',
   'Bye! Come back again soon.']},
 {'tag': 'thanks',
  'patterns': ['Thanks', 'Thank you', "That's helpful"],
  'responses': ['Happy to help!', 'Any time!', 'My pleasure']}]

### Block 2 Code Description
In this code block we are reading the data from the file using json tool, Json is an easy light weight key value storage file system.
By using this system the file size decreases drastically and query to a json file will be lightning fast when we have lots of data.

In [9]:
# Code block 3 
# Empty lists for appending the data after processing NLP
words=[]
documents = []
classes = []

# This list will be used for ignoring all unwanted punctuation marks.
ignore = ["?"]

# Starting a loop through each intent in intents["patterns"]
for intent in intents["intents"]: # looping through each intent
    for pattern in intent["patterns"]: # picking through each pattern from intent
        
        # tokenizing each and every word in the sentence by using word tokenizer and storing in w
        # tokenizing the each pattern_word using tokenizer
        w = nltk.word_tokenize(pattern) 
        #print(w)
        
        # Adding tokenized pattern words to words empty list that we defined above
        words.extend(w) 
        #print(words)
        
        # Adding words to documents with tag given in intents file as a tuple format
        documents.append((w, intent["tag"]))
        #print(documents)
        
        # Adding only tag to our classes list
        if intent["tag"] not in classes:      
            classes.append(intent["tag"])  #If tag is not present in classes[] then it will append into it.
            #print(classes)
            

#### Block 3 code explanation
In this code block we are generating three lists namely words which contains tokens of the patterns, classes which contains the tag of the particular patterns, and document which containes the pattern and class. For this operation we are using for loop to iterate through the datafile and generate the tokens, pick the classes of each pattern.  

In [10]:
# Code Block 4

#Performing Stemming by using stemmer.stem() on lowercase of each word 
#Running loop in words[] and ignoring punctuation marks present in ignore[]
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]  
words = sorted(list(set(words)))  #Removing Duplicates in words[]

#Removing Duplicate Classes
classes = sorted(list(set(classes)))

#Printing length of lists we formed
print(len(documents),"Documents \n")
print(len(classes),"Classes \n")
print(len(words), "Stemmed Words ")

88 Documents 

33 Classes 

97 Stemmed Words 


In [17]:
documents[0:3] , words[0:5]

([(['Hi'], 'greeting'),
  (['How', 'are', 'you'], 'greeting'),
  (['Is', 'anyone', 'there', '?'], 'greeting')],
 ["'s", 'a', 'act', 'an', 'and'])

### Block code 4 explanation
In this code block we are deriving the stem words and storing them in words variable and removing the duplicates of the stemmed words. For this task we are utlising nltk library's stemmer package and set data structure. Finally we are knowing the statistics of the documents, classess, and words. 

In [18]:
#Block code 5 Explanation

#Creating Training Data which will be furthur used for training
training = []
output = []

#Creating empty array for output
output_empty = [0] * len(classes)

#Creating Training set and bag of words for each sentence
for doc in documents:
    bag = [] #Initialising empty bag of words

    pattern_words = doc[0] #Storing list of tokenized words for the documents[] tp pattern_words
    #print(pattern_words)
    
    #Again Stemming each word from pattern_words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]  
    #print(pattern_words)
    
    #Creating bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    #It will give output 1 for curent tag and 0 for all other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] =1 
    training.append([bag, output_row])

### Block code 5 Explanation
In this code block we are generating data which is used to train the 
chatbot so that it can learn the patterns. For generating the training data we are utilising our documents variable and iterating it using a for loop, Also for every word in words variable we are performing stemming and checking if the word happens to be in our actual words from document to assign polarity to the particular tag using if else logic. Finally we are generating an array of 1's and 0's and appending it to the training variable that we defined earlier.  

In [22]:
# Code Block 6
random.shuffle(training) #Suffling the data or features
training = np.array(training) #Converting training data into numpy array

#Creating Training Lists
train_x = list(training[:,0])
train_y = list(training[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


### Block code 6 explanation
In this code block we are shuffling the training data and converting the data to an array with the help of numpy tool that we have imported earler. For any ML model we need two sets of data one is training data and another is label data so that the model can learn the pattern, For this we are breaking the shuffled training data into two parts using subsetting feature of an list data structure. 

In [23]:
# Code Block 7 
tf.reset_default_graph() #Reset Underlying Graph data

#Building our own Neural Network
net = tflearn.input_data(shape=[None, len(train_x[0])]) #adding an input layer to our neuralnetwork
net = tflearn.fully_connected(net, 10) #adding fully connected layer with 10 hidden neurons
net = tflearn.fully_connected(net, 10) #adding fully connected layer with 10 hidden neurons
net = tflearn.fully_connected(net, len(train_y[0]), activation="softmax") #adding a fully connected layer with activation function as softmax
net = tflearn.regression(net) #adding regression (linear or logistic) to the provided input.

#Defining Model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir="tflearn_logs")

#Now we have setup model, now we need to train that model by fitting data into it by model.fit()
#n_epoch is the number of times that model will se our data during training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)  # training the model using fit method with epoch as 1000 and batch size as 8
model.save("model.tflearn") #Saving the model

Training Step: 10999  | total loss: 0.46801 | time: 0.056s
| Adam | epoch: 1000 | loss: 0.46801 - acc: 0.9811 -- iter: 80/88
Training Step: 11000  | total loss: 0.42148 | time: 0.060s
| Adam | epoch: 1000 | loss: 0.42148 - acc: 0.9829 -- iter: 88/88
--


### Block code 7 explaination
With the aid of the tflearn package that we previously loaded, we are creating our own deep neural network in this code block. This network consists of three fully connected layers: an input layer, a regression layer, and a regression layer with customized parameters for two of the levels. For the output propagation based on a mathematical function, there is an activation function called softmax. Finally, in order to train on the training data and reduce loss (misinterpretations/error), we are employing this specially built feedforward neural network. We use the network.fit() method for the training, and then we save the model so that we can use it in the future.

In [24]:
# Code Block 8 

#Importing pickle module
import pickle

#Dumping training data by using dump() and writing it into training_data in binary mode
pickle.dump({"words":words, "classes":classes, "train_x":train_x, "train_y":train_y}, open("training_data", "wb"))

### Block code 8 explanation
In this code block we are saving all our variables as binarys using pickle package that we have imported earlier. The advatage of saving them is for re-usibility and progress saving. 


In [25]:
# code block 9
#Restoring all data structure
data = pickle.load(open("training_data","rb")) #using pickle package to load the training data 
# subsetting all the required data from the data variable
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

### Block code 9 explanation

In this code block we are loading all our saved binaries for re-usage 

In [26]:
# Code block 10
with open("intents.json") as json_data:
    intents = json.load(json_data)  #Loading our json_data

### Block code 10 explanation
In this code block we are reading data the intents datafile with the help of json package

In [28]:
# Code block 11

# Loading the saved model
model.load("./model.tflearn") #Loading training model which we saved

## Block code 11 explanation
In this block of code we are loading the model back to the kernel for re-usability

In [29]:
#Code block 12

#Cleaning User Input
def clean_up_sentence(sentence):
    
    # Tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence) #Again tokenizing the sentence
    
    #Stemming each word from the user's input
    sentence_words= [stemmer.stem(word.lower()) for word in sentence_words]

    return sentence_words

#Returning bag of words array: 0 or 1 or each word in the bag that exists in as we have declared in above lines
def bow(sentence, words, show_details=False):
    
    #Tokenizing the user input
    sentence_words = clean_up_sentence(sentence)
    
    #Generating bag of words from the sentence that user entered
    bag = [0]*len(words) #intialising bag vector with 0's
    for s in sentence_words: #iterating through the sentence words
        for i,w in enumerate(words):
            if w == s: #checking if word is the same as the word from user's sentence
                bag[i] = 1  #if matched assigning value of 1
                if show_details:
                    print("Found in bag: %s"% w)
    return(np.array(bag)) #cnverting bag to array and returning it

## Block code 12 explanation
In this code block we are defining two utility functions for processing the sentence and converting the sentence to tokens using stemming and tokenization technique. Also, we are converting the senetence into an machine readable form by using bag of words technique which returns an array of 1 and 0 for any given sentence. Later these can be fed into the chatbot model for answering our queries/questions.

In [30]:
# Code block 13.

#Adding some context to the conversation for better results.

context = {} #Create a dictionary to hold user's context.

ERROR_THRESHOLD = 0.25
def classify(sentence):
    
    #Generating probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    
    #Filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    
    #Sorting by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    
    # return tuple of intent and probability
    return return_list

def query(sentence, userID='123', show_details=False):
    results = classify(sentence)
    
    #If we have a classification then find the matching intent tag
    if results:
        
        #Loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                
                #Find a tag matching the first result
                if i['tag'] == results[0][0]:
                    
                    #Set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        
                        #A random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0) #sending out the response

# Block code 13 Explanation 
In this code block we are adding the contextual meaning to the model using fucntions. We will be using two fucntions, One does the job of classifying the input sentence into the tags and returns the sorted list of words by probability with the help of the trained model, Second fucntion returns the query result by identifying the tag from the results and filters the context using bunch of if else statements and gives us the response of the query. 

In [33]:
query("what is data") #testing the bot

Decryption is the process of taking encoded or encrypted text or other data and converting it back into text that you or the computer can read and understand. This term could be used to describe a method of unencrypting the data manually or unencrypting the data using the proper codes or keys.


In [37]:
query("Hi") #testing the bot

Hi there, how can I help?


In [39]:
query("I want to learn about vpn") #testing the bot

VPN stands for Virtual Private Network. It is used to create a safe and encrypted connection. When you use a VPN, the data from the client is sent to a point in the VPN where it is encrypted and then sent through the internet to another point. At this point, the data is decrypted and sent to the server. When the server sends a response, the response is sent to a point in the VPN where it is encrypted and this encrypted data is sent to another point in the VPN where it is decrypted. And finally, the decrypted data is sent to the client. The whole point of using a VPN is to ensure encrypted data transfer.


In [42]:
query("hat means hackers grey ??") #testing the bot

Grey hat hackers are an amalgamation of a white hat and black hat hacker. They look for system vulnerabilities without the owner�s permission. If they find any vulnerabilities, they report it to the owner. Unlike Black hat hackers, they do not exploit the vulnerabilities found.


In [43]:
query("how is ur day going on") #testing the bot

Good to see you again


In [45]:
query("grey and vpn") #testing the bot

Traceroute is a tool that shows the path of a packet. It lists all the points (mainly routers) that the packet passes through. This is used mostly when the packet is not reaching its destination. Traceroute is used to check where the connection stops or breaks to identify the point of failure.


In [35]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'hours',
   'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?',
    'When is the time to contact ?',
    'At what time do you provide services ?'],
   'responses': ["We're open every day from 9AM to 9PM",
    'Our working hours are 9AM to 9PM every day']},
  {'tag': 'Cryptography',
   'patterns': ['What is mean by cryptography?', 'What is Cryptography?'],
   'responses'